**Задача**

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Описание данных**

[Схема данных](https://drive.google.com/file/d/1ZtCiLsLGhqNj7TRCieDatuaq1URPrhQ_/view?usp=sharing)

Таблица books

Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

Таблица authors

Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

Таблица publishers

Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

Таблица ratings

Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Таблица reviews

Содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

## Исследование таблиц

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text

In [2]:
# устанавливаем параметры

db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
query = """SELECT *
FROM books
LIMIT 5"""
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = """SELECT *
FROM authors
LIMIT 5"""
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query = """SELECT *
FROM publishers
LIMIT 5"""
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
query = """SELECT *
FROM ratings
LIMIT 5"""
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
query = """SELECT *
FROM reviews
LIMIT 5"""
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## SQL-запросы для решения заданий

### Посчитайте, сколько книг вышло после 1 января 2000 года

In [8]:
query = """SELECT COUNT(*)
FROM books
WHERE CAST(publication_date AS DATE) > '2000-01-01'"""
pd.io.sql.read_sql(query, con = engine)

,count
0,819


### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [14]:
query = """WITH
qq AS (SELECT ra.book_id,
COUNT(DISTINCT re.review_id) as number_of_reviews,
ROUND(AVG(ra.rating), 2) as avg_rating
FROM ratings AS ra
FULL OUTER JOIN reviews AS re ON ra.book_id = re.book_id
GROUP BY ra.book_id
ORDER BY ra.book_id)

SELECT title,
qq.number_of_reviews,
qq.avg_rating
FROM books AS b
RIGHT JOIN qq ON b.book_id = qq.book_id
"""
pd.io.sql.read_sql(query, con = engine)

,title,number_of_reviews,avg_rating
0,'Salem's Lot,2,3.67
1,1 000 Places to See Before You Die,1,2.50
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.67
3,1491: New Revelations of the Americas Before C...,2,4.50
4,1776,4,4.00
...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),3,3.67
996,Xenocide (Ender's Saga #3),3,3.40
997,Year of Wonders,4,3.20
998,You Suck (A Love Story #2),2,4.50


### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [10]:
query = """WITH
b AS (SELECT publisher_id
FROM books
WHERE num_pages > 50
GROUP BY publisher_id
ORDER BY COUNT(book_id) DESC
LIMIT 1)

SELECT publisher
FROM publishers as p
RIGHT JOIN b ON b.publisher_id = p.publisher_id
"""
pd.io.sql.read_sql(query, con = engine)

,publisher
0,Penguin Books


### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [16]:
query = """WITH
b AS (SELECT book_id,
COUNT(rating) as number_of_ratings,
AVG(rating) as avg_rating
FROM ratings
GROUP BY book_id
HAVING COUNT(rating) >= 50)

SELECT author,
AVG(avg_rating)
FROM b
LEFT JOIN books ON b.book_id = books.book_id
LEFT JOIN authors as a ON books.author_id = a.author_id
GROUP BY author
ORDER BY avg DESC
LIMIT 1
"""
pd.io.sql.read_sql(query, con = engine)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [12]:
query = """WITH
ra AS (SELECT username,
COUNT(rating) AS number_of_ratings
FROM ratings
GROUP BY username
HAVING COUNT(rating) > 50),

re AS (SELECT username,
COUNT(review_id)number_of_reviews
FROM reviews
GROUP BY username)

SELECT AVG(number_of_reviews)
FROM ra
LEFT JOIN re ON ra.username = re.username

"""
pd.io.sql.read_sql(query, con = engine)

,avg
0,24.333333
